In [11]:
import cobra
from cobra.io import save_json_model, read_sbml_model, write_sbml_model
import os, re
import pandas as pd

print(os.getcwd())

ext_dir = '/../../../data/external'
gap_uni_dir = '/../2.gapfill_w_universal/Salb-GEM-Uni-gapfill.xml'

# Import model
# model = read_sbml_model(f"{os.getcwd()}/{ext_dir}/Salb-GEM.xml")
# model = read_sbml_model(f"{os.getcwd()}/{ext_dir}/Salb-GEM-Updated.xml")
model = read_sbml_model(f"{os.getcwd()}/{gap_uni_dir}")
model_ori = read_sbml_model(f"{os.getcwd()}/{ext_dir}/Salb-GEM-Updated.xml")

/Users/te/Library/CloudStorage/OneDrive-Personal/Biosustain/strep/Code/streptsd/notebooks/gapfilling/3.manual_check


'' is not a valid SBML 'SId'.
'' is not a valid SBML 'SId'.


In [22]:
model.reactions.NRRH

Reaction identifier,NRRH
Name,nicotinate D-ribonucleoside ribohydrolase
Memory address,0x7f9d64bb7f50
Stoichiometry,h2o_c + nicrns_c --> h_c + ncam_c + rib__D_c H2O + nicotinate nucleoside --> H+ + Nicotinamide + D-Ribose
GPR,
Lower bound,0.0
Upper bound,1000.0


In [25]:
for r in model.metabolites.ncam_c.reactions:
    print(r.build_reaction_string())

h_c + ncam_c + r1p_c <=> pi_c + rnam_c
h2o_c + nicrns_c --> h_c + ncam_c + rib__D_c
h2o_c + ncam_c <=> nac_c + nh4_c
nicrns_c + pi_c --> h_c + ncam_c + r1p_c


In [26]:
model.metabolites.nac_c

Metabolite identifier,nac_c
Name,Nicotinate
Memory address,0x7f9d64761090
Formula,C6H4NO2
Compartment,c
In 6 reaction(s),"NNAM, NCTPPRT, NAMNPP, NNDMBRT, NACt, NP1"


In [12]:
with model:
    print(model.slim_optimize())

0.1626633553770976


# Get all gapfilled reactions

In [13]:
diff_r = []
for r in model.reactions:
    if model_ori.reactions.has_id(r.id) == False:
        diff_r.append(r)

reaction_data = [{
    'Reaction ID': reaction.id,
    'Name': reaction.name,
    'Reaction String': reaction.reaction,
    'Reference': reaction.annotation
} for reaction in diff_r]

reaction_df = pd.DataFrame(reaction_data)
reaction_df.to_csv('all_gapfilled_reactions.csv')
print(reaction_df)

     Reaction ID                                               Name  \
0           HEX4                         hexokinase (D-mannose:ATP)   
1           FBA4  L-xylulose 1-phosphate glycolaldehyde-lyase (g...   
2        PPRDNDH                          Piperideine dehydrogenase   
3        2METBOD                        2-methylisoborneol synthase   
4         NO2t2r  nitrite transport in via proton symport, rever...   
5        5DGLCNR                    5-dehydro-D-gluconate reductase   
6         GERDPM          geranyl diphosphate 2-C-methyltransferase   
7       EX_h2s_e                          Hydrogen sulfide exchange   
8         MACACI                       maleylacetoacetate isomerase   
9          H2St1                                      h2s transport   
10        ACONIs                  aconitate isomerase (spontaneous)   
11        APTNAT  5-aminopentanoate:2-oxoglutarate aminotransferase   
12        MANpts                    D-mannose transport via PEP:PTS   
13  EX

In [14]:
# Find out what genes could be wrong by checking growth.

possibly_wrong = []
for r in diff_r:
    with model_ori:
        model_ori.add_reactions([r])
        model_ori.objective = 'growth'
        solution = model_ori.optimize().objective_value
    print(r, solution)
    if solution > 0.1:
        possibly_wrong.append(r)

possibly_wrong

HEX4: atp_c + man_c --> adp_c + h_c + man6p_c 0.05217336185319639
FBA4: xu1p__D_c <=> dhap_c + gcald_c 0.05217336185319648
PPRDNDH: 2.0 h2o_c + nad_c + pprdn_c --> 5aptn_c + h_c + nadh_c 0.052173361853197375
2METBOD: 2mygydpp_c + h2o_c --> 2mborn_c + ppi_c 0.05217336185319693
NO2t2r: h_e + no2_e <=> h_c + no2_c 0.05217336185319693
5DGLCNR: 5dglcn_c + h_c + nadph_c <=> glcn_c + nadp_c 0.05217336185319693
GERDPM: amet_c + grdp_c --> 2mygydpp_c + ahcys_c + h_c 0.052173361853197105
EX_h2s_e: h2s_e -->  0.05217336185319689
MACACI: 4mlacac_c <=> 4fumacac_c 0.05217336185319689
H2St1: h2s_e <=> h2s_c 0.052173361853197375
ACONIs: acon_T_c <=> acon_C_c 0.05217336185319693
APTNAT: 5aptn_c + akg_c <=> glu__L_c + oxptn_c 0.052173361853197375
MANpts: man_e + pep_c --> man6p_c + pyr_c 0.052173361853197334
EX_citr__L_e: citr__L_e -->  0.052173361853195634
LYXI: lyx__L_c <=> xylu__L_c 0.05217336185319608
DM_2mborn_c: 2mborn_c -->  0.05217336185319686
CITRLt: citr__L_c <=> citr__L_e 0.05217336185319686


[<Reaction DTDP26DGR at 0x7f9d64ba0c90>,
 <Reaction DHPS2_2 at 0x7f9d64bb7050>,
 <Reaction SRK at 0x7f9d64bc1a50>,
 <Reaction SDPDS_2 at 0x7f9d64bc1bd0>,
 <Reaction THDPS_2 at 0x7f9d64bc6cd0>]

In [15]:
addition = list(set(diff_r) - set(possibly_wrong))

# Manual Check Gapfilled Reactions in all_gapfilled_reactions.xlsx

1. Possibly wrong reactions are removed from the list (DTDP26DGR, DHPS_2, SRK, SDPDS_2, THDPS_2)
2. Reaction names are updated
3. TODO: possibly matching a gene

In [16]:
# Update reaction names
for r in addition:
    if r.name == "":
        print(r.id)


CPS_1
NP1_2
NRRH


In [17]:
# Reaction name is updated from kegg and ncbi
# https://www.ncbi.nlm.nih.gov/datasets/taxonomy/1883
model.reactions.NP1_2.name = 'nicotinate D-ribonucleoside:orthophosphate ribosyltransferase'
model.reactions.NP1_2.annotation = {'kegg': 'R02295'}

model.reactions.CPS_1.name = 'carbamoyl-phosphate synthase'
model.reactions.CPS_1.annotation = {'kegg':'R00149'}

model.reactions.NRRH.name = 'nicotinate D-ribonucleoside ribohydrolase'
model.reactions.NRRH.annotation = {'kegg': 'R10046'}

In [21]:
with model_ori:
    model_ori.add_reactions(addition)
    media = model_ori.medium
    media['EX_glc__D_e'] = 0
    # media['EX_mnl_e'] = 0.8
    media['EX_glu__L_e'] = 0.96
    # media['EX_glyc_e'] = 1.6
    model_ori.medium = media
    model.objective = 'growth'
    solution = model_ori.optimize()

print(solution.objective_value)

Ignoring reaction 'HEX4' since it already exists.
Ignoring reaction '5DGLCNR' since it already exists.
Ignoring reaction 'GERDPM' since it already exists.
Ignoring reaction 'EX_citr__L_e' since it already exists.
Ignoring reaction 'MANpts' since it already exists.
Ignoring reaction 'GLYC3tr' since it already exists.
Ignoring reaction 'DAPAT' since it already exists.
Ignoring reaction 'ARAI' since it already exists.
Ignoring reaction 'CPS_1' since it already exists.
Ignoring reaction 'PHYCBOXL' since it already exists.
Ignoring reaction 'ETHAAL' since it already exists.
Ignoring reaction 'ACGApts' since it already exists.
Ignoring reaction 'NP1_2' since it already exists.
Ignoring reaction 'NO2t2r' since it already exists.
Ignoring reaction 'GAMpts' since it already exists.
Ignoring reaction 'HMR_9726' since it already exists.
Ignoring reaction 'r1891' since it already exists.
Ignoring reaction 'CITRLt' since it already exists.
Ignoring reaction 'SALCpts' since it already exists.
Ignori

0.040557031263983905


In [19]:
model_ori.add_reactions(addition)

In [20]:
model_ori.reactions.EX_co2_e.bounds= (0, 1000)

cobra.io.write_sbml_model(model_ori, 'Salb-GEM-Man-gapfill.xml')